In [1]:
from itertools import permutations

import pandas as pd

from glicko_rating_matches import GlickoSystem, Team
from lol_fandom import get_tournaments, get_match_schedule

pd.set_option("display.max_columns", None)

In [2]:
def get_team_id(teams_id, team_name):
    return teams_id.loc[teams_id["team"] == team_name, "team_id"].iloc[0]

In [3]:
teams_id = pd.read_csv("./csv/teams_id.csv")
ratings = pd.read_csv(
    "./csv/glicko_rating/glicko_rating_2023.csv", parse_dates=["last_game_date"]
)
teams_id.shape, ratings.shape

((755, 2), (597, 15))

In [4]:
ratings.loc[(ratings["league"] == "LCK") & (ratings["last_game_date"].dt.year == 2023)]

,team_id,team,league,games,win,loss,winrate,streak,point,rd,sigma,error,error_square,last_game_date,change
1,220,Gen.G,LCK,1018,608,410,0.597250,1,2015.035095,72.582396,0.586825,431.794816,229.218906,2023-06-11 11:26:00,- (-)
2,69,T1,LCK,1440,954,486,0.662500,-1,1989.937628,72.553807,0.585984,570.130589,301.349939,2023-06-11 11:26:00,- (-)
4,219,KT Rolster,LCK,1051,622,429,0.591817,-2,1919.579768,72.365438,0.588215,437.041153,232.171357,2023-06-09 10:24:00,- (-)
6,508,Dplus KIA,LCK,622,437,185,0.702572,1,1886.968891,62.214323,0.588338,230.453718,120.364050,2023-06-11 07:52:00,- (-)
11,205,Hanwha Life Esports,LCK,859,425,434,0.494761,-2,1803.704657,65.729225,0.587499,365.359084,192.772994,2023-06-09 13:52:00,- (-)
20,509,Liiv SANDBOX,LCK,449,215,234,0.478842,2,1752.096369,67.852364,0.592067,198.656338,104.652186,2023-06-10 07:08:00,- (-)
25,301,Kwangdong Freecs,LCK,905,430,475,0.475138,4,1739.402939,68.575378,0.592286,404.657779,216.367474,2023-06-10 09:36:00,- (-)
64,33,DRX,LCK,1103,572,531,0.518586,-1,1652.135967,62.218026,0.588501,441.272653,226.673257,2023-06-11 07:52:00,-1 (-)
116,524,Nongshim RedForce,LCK,302,116,186,0.384106,-2,1580.762195,67.785024,0.589673,129.483411,69.823872,2023-06-10 07:08:00,- (-)
117,15,OKSavingsBank BRION,LCK,492,177,315,0.359756,-4,1578.227817,68.490002,0.589739,203.229720,108.733392,2023-06-10 09:36:00,- (-)


In [5]:
ratings.loc[(ratings["league"] == "LPL") & (ratings["last_game_date"].dt.year == 2023)]

,team_id,team,league,games,win,loss,winrate,streak,point,rd,sigma,error,error_square,last_game_date,change
0,444,JD Gaming,LPL,737,436,301,0.591588,6,2032.352198,54.658744,0.557087,301.545426,154.331899,2023-06-13 12:10:00,- (+5)
3,458,Bilibili Gaming,LPL,550,287,263,0.521818,1,1975.981588,40.759244,0.556284,250.045052,131.324100,2023-06-12 14:08:00,- (-)
10,170,EDward Gaming,LPL,1201,740,461,0.616153,-3,1817.767862,64.247015,0.575162,500.171633,263.420419,2023-06-10 12:07:00,- (-)
13,244,LNG Esports,LPL,836,419,417,0.501196,1,1795.227909,62.048403,0.567565,385.850996,205.664952,2023-06-11 09:03:00,- (-)
15,461,Top Esports,LPL,691,398,293,0.575977,2,1792.894110,71.835810,0.566143,280.419486,143.125341,2023-06-11 13:53:00,- (-)
17,445,Weibo Gaming,LPL,567,308,259,0.543210,-2,1775.980313,72.170037,0.576697,258.620765,138.572579,2023-06-11 13:53:00,- (-)
18,60,Oh My God,LPL,1042,480,562,0.460653,2,1769.861722,69.406478,0.568251,441.610198,226.289511,2023-06-13 10:47:00,- (+2)
31,305,Royal Never Give Up,LPL,1154,744,410,0.644714,-4,1718.013742,54.934948,0.572873,477.631877,252.691278,2023-06-13 12:10:00,-2 (-5)
50,39,Team WE,LPL,1113,584,529,0.524708,6,1671.546975,71.414374,0.586273,500.361191,262.564310,2023-06-10 10:14:00,- (-)
72,515,Ninjas in Pyjamas.CN,LPL,378,161,217,0.425926,-2,1638.603904,69.342852,0.566094,165.133862,89.598490,2023-06-13 10:47:00,- (-2)


In [6]:
ratings.loc[(ratings["league"] == "LEC") & (ratings["last_game_date"].dt.year == 2023)]

,team_id,team,league,games,win,loss,winrate,streak,point,rd,sigma,error,error_square,last_game_date,change
7,211,G2 Esports,LEC,1013,578,435,0.570582,-2,1861.413869,71.846024,0.564581,399.603502,209.606027,2023-05-16 15:11:00,- (-)
41,333,MAD Lions,LEC,676,334,342,0.494083,-6,1695.711851,65.737715,0.584468,301.907087,160.869155,2023-05-13 17:05:00,- (-)
47,598,Team BDS,LEC,72,30,42,0.416667,-3,1680.847413,72.360829,0.582511,30.183166,16.469519,2023-04-23 19:40:00,- (-)
73,334,Team Vitality,LEC,386,172,214,0.445596,-2,1637.948485,72.005713,0.584933,172.738544,91.498237,2023-04-22 18:48:00,- (-)
96,3,Fnatic,LEC,1243,758,485,0.609815,-1,1605.252338,72.620249,0.587580,512.537617,271.612358,2023-04-15 17:52:00,- (-)
118,611,KOI (Spanish Team),LEC,38,17,21,0.447368,-2,1577.540826,60.766348,0.579427,18.129202,10.189942,2023-04-16 19:37:00,- (-)
120,310,Astralis,LEC,457,225,232,0.492341,-4,1574.429320,71.708896,0.584751,182.310327,92.462197,2023-04-16 17:06:00,- (-)
148,42,SK Gaming,LEC,395,184,211,0.465823,-7,1546.525845,57.558267,0.591374,186.176098,99.867698,2023-04-15 19:46:00,-1 (-)
199,518,Excel Esports,LEC,173,64,109,0.369942,-4,1514.252750,72.256232,0.578864,75.091036,40.440209,2023-03-27 15:58:00,- (-)
280,612,Team Heretics,LEC,24,8,16,0.333333,-1,1478.673501,66.041777,0.575617,11.151701,5.991388,2023-03-27 21:00:00,- (-)


In [7]:
ratings.loc[(ratings["league"] == "LCS") & (ratings["last_game_date"].dt.year == 2023)]

,team_id,team,league,games,win,loss,winrate,streak,point,rd,sigma,error,error_square,last_game_date,change
16,86,Cloud9,LCS,1085,605,480,0.557604,-3,1789.300717,59.941851,0.574980,451.606554,233.143856,2023-05-17 13:43:00,- (-)
29,470,Golden Guardians,LCS,287,111,176,0.386760,-1,1721.928003,70.876570,0.558671,127.430448,67.987622,2023-05-14 16:36:00,+1 (-)
43,163,Evil Geniuses.NA,LCS,366,192,174,0.524590,-3,1691.744681,67.619417,0.585426,166.852126,87.636594,2023-03-31 22:45:00,- (-)
67,414,FlyQuest,LCS,417,190,227,0.455635,-2,1650.145277,70.888778,0.574269,194.618819,102.510722,2023-04-09 00:07:00,-1 (-)
86,264,Team Liquid,LCS,860,496,364,0.576744,1,1614.105039,66.932791,0.554640,373.434404,199.321935,2023-03-17 23:09:00,- (-)
103,468,100 Thieves,LCS,444,204,240,0.459459,-2,1598.070175,69.785580,0.582777,181.145054,89.932534,2023-03-26 00:23:00,+1 (-)
127,1,Counter Logic Gaming,LCS,696,336,360,0.482759,-5,1564.587089,67.193728,0.584382,321.127325,167.945502,2023-03-26 22:57:00,- (-)
149,4,TSM,LCS,1153,695,458,0.602775,-1,1546.476212,69.017374,0.555351,448.445440,236.088101,2023-03-17 22:08:00,-1 (-)
532,350,Immortals,LCS,337,166,171,0.492582,-2,1379.860678,56.593654,0.548881,147.025587,75.752154,2023-03-18 01:18:00,- (-)
548,31,Dignitas,LCS,489,206,283,0.421268,1,1369.315275,66.060757,0.548726,227.131221,119.054176,2023-03-18 00:15:00,- (-)


In [8]:
ratings.loc[
    (ratings["league"].isin(["LCK", "LPL", "LEC", "LCS"]))
    & (ratings["last_game_date"].dt.year == 2023)
]

,team_id,team,league,games,win,loss,winrate,streak,point,rd,sigma,error,error_square,last_game_date,change
0,444,JD Gaming,LPL,737,436,301,0.591588,6,2032.352198,54.658744,0.557087,301.545426,154.331899,2023-06-13 12:10:00,- (+5)
1,220,Gen.G,LCK,1018,608,410,0.597250,1,2015.035095,72.582396,0.586825,431.794816,229.218906,2023-06-11 11:26:00,- (-)
2,69,T1,LCK,1440,954,486,0.662500,-1,1989.937628,72.553807,0.585984,570.130589,301.349939,2023-06-11 11:26:00,- (-)
3,458,Bilibili Gaming,LPL,550,287,263,0.521818,1,1975.981588,40.759244,0.556284,250.045052,131.324100,2023-06-12 14:08:00,- (-)
4,219,KT Rolster,LCK,1051,622,429,0.591817,-2,1919.579768,72.365438,0.588215,437.041153,232.171357,2023-06-09 10:24:00,- (-)
6,508,Dplus KIA,LCK,622,437,185,0.702572,1,1886.968891,62.214323,0.588338,230.453718,120.364050,2023-06-11 07:52:00,- (-)
7,211,G2 Esports,LEC,1013,578,435,0.570582,-2,1861.413869,71.846024,0.564581,399.603502,209.606027,2023-05-16 15:11:00,- (-)
10,170,EDward Gaming,LPL,1201,740,461,0.616153,-3,1817.767862,64.247015,0.575162,500.171633,263.420419,2023-06-10 12:07:00,- (-)
11,205,Hanwha Life Esports,LCK,859,425,434,0.494761,-2,1803.704657,65.729225,0.587499,365.359084,192.772994,2023-06-09 13:52:00,- (-)
13,244,LNG Esports,LPL,836,419,417,0.501196,1,1795.227909,62.048403,0.567565,385.850996,205.664952,2023-06-11 09:03:00,- (-)


In [9]:
leagues = ["LCK", "LPL", "LEC", "LCS"]
for league in leagues:
    print(league)
    lst = ratings.loc[
        (ratings["league"] == league) & (ratings["last_game_date"].dt.year == 2023),
        "team",
    ].unique()
    print(", ".join(map(lambda x: f"'{x}'", lst)))

LCK
'Gen.G', 'T1', 'KT Rolster', 'Dplus KIA', 'Hanwha Life Esports', 'Liiv SANDBOX', 'Kwangdong Freecs', 'DRX', 'Nongshim RedForce', 'OKSavingsBank BRION'
LPL
'JD Gaming', 'Bilibili Gaming', 'EDward Gaming', 'LNG Esports', 'Top Esports', 'Weibo Gaming', 'Oh My God', 'Royal Never Give Up', 'Team WE', 'Ninjas in Pyjamas.CN', 'Invictus Gaming', 'Rare Atom', 'FunPlus Phoenix', 'ThunderTalk Gaming', 'Ultra Prime', 'Anyone's Legend', 'LGD Gaming'
LEC
'G2 Esports', 'MAD Lions', 'Team BDS', 'Team Vitality', 'Fnatic', 'KOI (Spanish Team)', 'Astralis', 'SK Gaming', 'Excel Esports', 'Team Heretics'
LCS
'Cloud9', 'Golden Guardians', 'Evil Geniuses.NA', 'FlyQuest', 'Team Liquid', '100 Thieves', 'Counter Logic Gaming', 'TSM', 'Immortals', 'Dignitas'


In [10]:
leagues = {
    "LCK": "LoL Champions Korea",
    "LPL": "Tencent LoL Pro League",
    "LEC": "LoL EMEA Championship",
    "LCS": "League of Legends Championship Series",
    "MSI": "Mid-Season Invitational",
    "WCS": "World Championship",
}

tournaments = get_tournaments(where=f'T.Year=2023 and T.League="{leagues["LCK"]}"')
tournaments

,Name,OverviewPage,DateStart,Date,League,Region,EventType,StandardName,Split,SplitNumber,TournamentLevel,IsQualifier,IsPlayoffs,IsOfficial,Year,DateStart__precision,Date__precision
0,LCK 2023 Spring,LCK/2023 Season/Spring Season,2023-01-18,2023-03-19,LoL Champions Korea,Korea,None,LCK 2023 Spring,Spring,1,Primary,0,0,1,2023,1,1
1,LCK 2023 Spring Playoffs,LCK/2023 Season/Spring Playoffs,2023-03-22,2023-04-09,LoL Champions Korea,Korea,Offline,LCK 2023 Spring Playoffs,Spring,1,Primary,0,1,1,2023,1,1
2,LCK 2023 Summer,LCK/2023 Season/Summer Season,2023-06-07,2023-08-06,LoL Champions Korea,Korea,None,LCK 2023 Summer,Summer,2,Primary,0,0,1,2023,1,1
3,LCK 2023 Summer Playoffs,LCK/2023 Season/Summer Playoffs,2023-08-08,2023-08-20,LoL Champions Korea,Korea,Offline,LCK 2023 Summer Playoffs,Summer,2,Primary,0,1,1,2023,1,1


In [11]:
page = "LCK/2023 Season/Summer Season"
match_schedules = get_match_schedule(where=f'MS.OverviewPage="{page}"').sort_values(
    by=["DateTime UTC"], ignore_index=True
)


teams = {}
lst_team = []
for team_name in match_schedules[["Team1", "Team2"]].unstack().unique():
    team_id = get_team_id(teams_id, team_name)
    row = ratings.loc[ratings["team"] == team_name].iloc[0]
    teams[team_id] = Team(
        name=team_name,
        league=row["league"],
        team_id=row["team_id"],
        win=row["win"],
        loss=row["loss"],
        streak=row["streak"],
        point=row["point"],
        rd=row["rd"],
        sigma=row["sigma"],
        last_game_date=row["last_game_date"],
    )
    lst_team.append(teams[team_id])

In [12]:
match_schedules = match_schedules.loc[match_schedules["Winner"].isna()].reset_index(
    drop=True
)
match_schedules

,Team1,Team2,Team1Final,Team2Final,Winner,Team1Points,Team2Points,Team1PointsTB,Team2PointsTB,Team1Score,Team2Score,Team1Poster,Team2Poster,Team1Advantage,Team2Advantage,FF,IsNullified,Player1,Player2,MatchDay,DateTime UTC,HasTime,DST,IsFlexibleStart,IsReschedulable,OverrideAllowPredictions,OverrideDisallowPredictions,IsTiebreaker,OverviewPage,ShownName,ShownRound,BestOf,Round,Phase,N MatchInPage,Tab,N MatchInTab,N TabInPage,N Page,Patch,PatchPage,Hotfix,DisabledChampions,PatchFootnote,InitialN MatchInTab,InitialPageAndTab,GroupName,Stream,StreamDisplay,Venue,CastersPBP,CastersColor,Casters,MVP,MVPPoints,VodInterview,VodHighlights,InterviewWith,Recap,Reddit,QQ,Wanplus,WanplusId,PageAndTeam1,PageAndTeam2,Team1Footnote,Team2Footnote,Footnote,UniqueMatch,MatchId,DateTime UTC__precision
0,Hanwha Life Esports,DRX,Hanwha Life Esports,DRX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1,2023-06-14 08:00:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,11,Week 2,1,2,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Hanwha Life Esports,LCK/2023 Season/Summer Season_DRX,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 2_1,LCK/2023 Season/Summer Season_Week 2_1,0
1,Gen.G,Kwangdong Freecs,Gen.G,Kwangdong Freecs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1,2023-06-14 10:30:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,12,Week 2,2,2,1,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Gen.G,LCK/2023 Season/Summer Season_Kwangdong Freecs,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 2_2,LCK/2023 Season/Summer Season_Week 2_2,0
2,T1,Liiv SANDBOX,T1,Liiv SANDBOX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,2,2023-06-15 08:00:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,13,Week 2,3,2,1,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_T1,LCK/2023 Season/Summer Season_Liiv SANDBOX,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 2_3,LCK/2023 Season/Summer Season_Week 2_3,0
3,Dplus KIA,OKSavingsBank BRION,Dplus KIA,OKSavingsBank BRION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,2,2023-06-15 10:30:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,14,Week 2,4,2,1,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Dplus KIA,LCK/2023 Season/Summer Season_OKSavingsBank BRION,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 2_4,LCK/2023 Season/Summer Season_Week 2_4,0
4,Nongshim RedForce,Gen.G,Nongshim RedForce,Gen.G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,3,2023-06-16 08:00:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,15,Week 2,5,2,1,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Nongshim RedForce,LCK/2023 Season/Summer Season_Gen.G,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 2_5,LCK/2023 Season/Summer Season_Week 2_5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Nongshim RedForce,KT Rolster,Nongshim RedForce,KT Rolster,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,3,2023-08-04 10:30:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,46,Week 9,6,5,2,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [13]:
glicko = GlickoSystem()

In [14]:
matches = match_schedules.head(10)
matches

,Team1,Team2,Team1Final,Team2Final,Winner,Team1Points,Team2Points,Team1PointsTB,Team2PointsTB,Team1Score,Team2Score,Team1Poster,Team2Poster,Team1Advantage,Team2Advantage,FF,IsNullified,Player1,Player2,MatchDay,DateTime UTC,HasTime,DST,IsFlexibleStart,IsReschedulable,OverrideAllowPredictions,OverrideDisallowPredictions,IsTiebreaker,OverviewPage,ShownName,ShownRound,BestOf,Round,Phase,N MatchInPage,Tab,N MatchInTab,N TabInPage,N Page,Patch,PatchPage,Hotfix,DisabledChampions,PatchFootnote,InitialN MatchInTab,InitialPageAndTab,GroupName,Stream,StreamDisplay,Venue,CastersPBP,CastersColor,Casters,MVP,MVPPoints,VodInterview,VodHighlights,InterviewWith,Recap,Reddit,QQ,Wanplus,WanplusId,PageAndTeam1,PageAndTeam2,Team1Footnote,Team2Footnote,Footnote,UniqueMatch,MatchId,DateTime UTC__precision
0,Hanwha Life Esports,DRX,Hanwha Life Esports,DRX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1,2023-06-14 08:00:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,11,Week 2,1,2,1,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Hanwha Life Esports,LCK/2023 Season/Summer Season_DRX,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 2_1,LCK/2023 Season/Summer Season_Week 2_1,0
1,Gen.G,Kwangdong Freecs,Gen.G,Kwangdong Freecs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1,2023-06-14 10:30:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,12,Week 2,2,2,1,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Gen.G,LCK/2023 Season/Summer Season_Kwangdong Freecs,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 2_2,LCK/2023 Season/Summer Season_Week 2_2,0
2,T1,Liiv SANDBOX,T1,Liiv SANDBOX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,2,2023-06-15 08:00:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,13,Week 2,3,2,1,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_T1,LCK/2023 Season/Summer Season_Liiv SANDBOX,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 2_3,LCK/2023 Season/Summer Season_Week 2_3,0
3,Dplus KIA,OKSavingsBank BRION,Dplus KIA,OKSavingsBank BRION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,2,2023-06-15 10:30:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,14,Week 2,4,2,1,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Dplus KIA,LCK/2023 Season/Summer Season_OKSavingsBank BRION,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 2_4,LCK/2023 Season/Summer Season_Week 2_4,0
4,Nongshim RedForce,Gen.G,Nongshim RedForce,Gen.G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,3,2023-06-16 08:00:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,15,Week 2,5,2,1,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Nongshim RedForce,LCK/2023 Season/Summer Season_Gen.G,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 2_5,LCK/2023 Season/Summer Season_Week 2_5,0
5,DRX,KT Rolster,DRX,KT Rolster,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,3,2023-06-16 10:30:00,1,yes,0,NaN,0,0,0,LCK/2023 Season/Summer Season,LCK 2023 Summer,NaN,3,NaN,NaN,16,Week 2,6,2,1,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,https://www.twitch.tv/lck,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LCK/2023 Season/Summer Season_DRX,LCK/2023 Season/Summer Season_KT Rolster,NaN,NaN,NaN,LCK/2023 Season/Summer Season_Week 2_6,LCK/2023 Season/Summer Season_Week 2_6,0
6,Hanwha Life Esports,Kwangdong Freecs,Hanwha Life Esports,Kwangdong Freecs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,4,2023-06-17

In [15]:
columns = ["Team1", "Point1", "Winprob1", "Winprob2", "Point2", "Team2"]
probs = pd.DataFrame(columns=columns)
for row in matches.itertuples():
    team1_name = row.Team1
    team2_name = row.Team2
    team1 = teams[get_team_id(teams_id, team1_name)]
    team2 = teams[get_team_id(teams_id, team2_name)]
    lst = [
        team1.name,
        team1.point,
        team1.get_win_probability(glicko, team2) * 100,
        team2.get_win_probability(glicko, team1) * 100,
        team2.point,
        team2.name,
    ]
    df = pd.DataFrame(data=[lst], columns=columns)
    probs = pd.concat([probs, df], ignore_index=True)
probs

,Team1,Point1,Winprob1,Winprob2,Point2,Team2
0,Hanwha Life Esports,1803.704657,70.181828,29.857066,1652.135967,DRX
1,Gen.G,2015.035095,82.496763,17.563959,1739.402939,Kwangdong Freecs
2,T1,1989.937628,79.223367,20.846382,1752.096369,Liiv SANDBOX
3,Dplus KIA,1886.968891,85.026764,14.885712,1578.227817,OKSavingsBank BRION
4,Nongshim RedForce,1580.762195,8.046264,92.011872,2015.035095,Gen.G
5,DRX,1652.135967,18.235597,81.911550,1919.579768,KT Rolster
6,Hanwha Life Esports,1803.704657,58.944795,41.039073,1739.402939,Kwangdong Freecs
7,OKSavingsBank BRION,1578.227817,9.033473,91.018631,1989.937628,T1
8,Liiv SANDBOX,1752.096369,28.097132,71.960563,1919.579768,KT Rolster
9,Dplus KIA,1886.968891,84.854594,15.067965,1580.762195,Nongshim RedForce


In [16]:
matches_bo3 = matches.loc[matches["BestOf"] == "3"]
columns = ["Team1", "Point1", "2:0", "2:1", "1:2", "0:2", "Point2", "Team2"]
probs = pd.DataFrame(columns=columns)
for row in matches_bo3.itertuples():
    team1_name = row.Team1
    team2_name = row.Team2
    team1 = teams[get_team_id(teams_id, team1_name)]
    team2 = teams[get_team_id(teams_id, team2_name)]
    team1_prob = team1.get_win_probability(glicko, team2)
    team2_prob = team2.get_win_probability(glicko, team1)
    lst = [
        team1.name,
        team1.point,
        team1_prob**2 * 100,
        (team1_prob**2) * team2_prob * 2 * 100,
        (team2_prob**2) * team1_prob * 2 * 100,
        team2_prob**2 * 100,
        team2.point,
        team2_name,
    ]
    df = pd.DataFrame(data=[lst], columns=columns)
    probs = pd.concat([probs, df], ignore_index=True)
probs["Winprob1"] = probs[["2:0", "2:1"]].sum(axis=1)
probs["Winprob2"] = probs[["1:2", "0:2"]].sum(axis=1)
columns = [
    "Team1",
    "Point1",
    "Winprob1",
    "2:0",
    "2:1",
    "1:2",
    "0:2",
    "Winprob2",
    "Point2",
    "Team2",
]
probs[columns]

,Team1,Point1,Winprob1,2:0,2:1,1:2,0:2,Winprob2,Point2,Team2
0,Hanwha Life Esports,1803.704657,78.667021,49.254890,29.412130,12.512640,8.914444,21.427084,1652.135967,DRX
1,Gen.G,2015.035095,91.964223,68.057160,23.907063,5.089929,3.084926,8.174855,1739.402939,Kwangdong Freecs
2,T1,1989.937628,88.931222,62.763418,26.167804,6.885646,4.345716,11.231362,1752.096369,Liiv SANDBOX
3,Dplus KIA,1886.968891,93.818906,72.295506,21.523401,3.768121,2.215844,5.983965,1578.227817,OKSavingsBank BRION
4,Nongshim RedForce,1580.762195,1.838837,0.647424,1.191413,13.624231,84.661845,98.286076,2015.035095,Gen.G
5,DRX,1652.135967,8.773095,3.325370,5.447724,24.470355,67.095019,91.565375,1919.579768,KT Rolster
6,Hanwha Life Esports,1803.704657,63.262850,34.744889,28.517961,19.855030,16.842055,36.697086,1739.402939,Kwangdong Freecs
7,OKSavingsBank BRION,1578.227817,2.301527,0.816036,1.485490,14.967366,82.843912,97.811278,1989.937628,T1
8,Liiv SANDBOX,1752.096369,19.256324,7.894488,11.361836,29.099202,51.783226,80.882428,1919.579768,KT Rolster
9,Dplus KIA,1886.968891,93.701801,72.003022,21.698780,3.853138,2.270436,6.123573,1580.762195,Nongshim RedForce


In [17]:
matches_bo5 = matches.loc[matches["BestOf"] == "5"]
columns = [
    "Team1",
    "Point1",
    "Winprob1",
    "3:0",
    "3:1",
    "3:2",
    "2:3",
    "1:3",
    "0:3",
    "Winprob2",
    "Point2",
    "Team2",
]
probs = pd.DataFrame(columns=columns)
for row in matches_bo5.itertuples():
    team1_name = row.Team1
    team2_name = row.Team2
    team1 = teams[get_team_id(teams_id, team1_name)]
    team2 = teams[get_team_id(teams_id, team2_name)]
    team1_prob = team1.get_win_probability(glicko, team2)
    team2_prob = team2.get_win_probability(glicko, team1)
    lst = [
        team1.name,
        team1.point,
        0,
        team1_prob**3 * 100,
        (team1_prob**3) * team2_prob * 3 * 100,
        (team1_prob**3) * (team2_prob**2) * 6 * 100,
        (team2_prob**3) * (team1_prob**2) * 6 * 100,
        (team2_prob**3) * team1_prob * 3 * 100,
        team2_prob**3 * 100,
        0,
        team2.point,
        team2.name,
    ]
    df = pd.DataFrame(data=[lst], columns=columns)
    probs = pd.concat([probs, df], ignore_index=True)
probs["Winprob1"] = probs[["3:0", "3:1", "3:2"]].sum(axis=1)
probs["Winprob2"] = probs[["2:3", "1:3", "0:3"]].sum(axis=1)
probs

,Team1,Point1,Winprob1,3:0,3:1,3:2,2:3,1:3,0:3,Winprob2,Point2,Team2
